In [10]:
from pprint import pprint
import pandas as pd
from gspread_dataframe import get_as_dataframe
import numpy as np

In [11]:
from utility import *
from gpt_method import *
from university import *
cache_repo_path = "./cache_repo"

In [12]:
# restore target_university sheet from local cache
restore_by_cache(os.path.join(cache_repo_path, "target_university_before_action.jsonl"), get_sheet_client())

Cache loaded successfully.


Worksheet 'target_university' found and deleted.
New 'target_university' worksheet created.
Data added to 'target_university' worksheet successfully.


In [13]:
# fill target_university worksheet and store it to local cache
output_cache = fill_target_university()
store_cache(os.path.join(cache_repo_path, "target_university_filled.jsonl"), output_cache)

In [14]:
# fetch the attribute format, reference, example, and prompts
attribute_sheet = get_worksheet("attribute_format")
attribute_columns = get_expect_column(sheet_title="attribute_format")
attribute_df = get_as_dataframe(attribute_sheet, evaluate_formulas=True, headers=True, usecols=attribute_columns)
attribute_df.head(17)

,attribute_name,attribute_format,attribute_reference,attribute_prompt,example
0,university_type,string,NaN,Focus on whether the university is public or p...,Public University
1,abbreviation,string,NaN,NaN,UBC
2,graduation_year,int,NaN,"The time for general students, undergraduate, ...",4
3,location,List[str],NaN,"The different locations of the campus, like di...",Vancouver campus\nOkanagan campus
4,graduation_rate,float,NaN,"Keep the format in two digits, like 89 represe...",NaN
5,domestic_student_tuition,string,https://universitystudy.ca/canadian-universities/,Visit https://universitystudy.ca/canadian-univ...,"""$6,128 - $7,946"""
6,international_student_tuition,string,https://universitystudy.ca/canadian-universities/,Visit https://universitystudy.ca/canadian-univ...,"""$37,038 - $51,448"""
7,description,string,NaN,You could try to extract information from the ...,"The University of British Columbia (UBC), loca..."
8,ranking,List[str],['https://www.topuniversities.com/world-univer...,You need to consider those four rankings for t...,2024 QS News | <ranking>\n2023 US News |<ranki...
9,website,string,NaN,Link of the official websites,https://www.ubc.ca/


In [15]:
# take target_university worksheet as a input
target_university_df = get_as_dataframe(
    get_worksheet(sheetname="target_university"),
    evaluate_formulas=True,
    headers=True,
    parse_dates=True,
    usecols=get_expect_column(sheet_title="target_university"),
)
target_university_df.head()


,ID,university_name,abbreviation,website,wikipedia
0,NaN,University of Waterloo,UW,https://uwaterloo.ca,https://en.wikipedia.org/wiki/University_of_Wa...
1,NaN,University of Toronto,U of T,https://www.utoronto.ca/,https://en.wikipedia.org/wiki/University_of_To...
2,NaN,Simon Fraser University,SFU,https://www.sfu.ca,https://en.wikipedia.org/wiki/Simon_Fraser_Uni...
3,NaN,Toronto Metropolitan University,TMU,https://www.metrou.ca,https://en.wikipedia.org/wiki/Toronto_Metropol...
4,NaN,NaN,NaN,NaN,NaN


In [16]:
expect_column = get_expect_column(sheet_title="university_template_en")
new_sheet = recreate_sheet(sheet_client=get_sheet_client(), title="output")
new_sheet.append_row(list(expect_column))  # Insert headers

Worksheet 'output' found and deleted.
New 'output' worksheet created.


{'spreadsheetId': '1CicjqDh97Ob4S339BG8jL0dRjtBug3q8AC2KiPrgHUE',
 'updates': {'spreadsheetId': '1CicjqDh97Ob4S339BG8jL0dRjtBug3q8AC2KiPrgHUE',
  'updatedRange': 'output!A1:S1',
  'updatedRows': 1,
  'updatedColumns': 19,
  'updatedCells': 19}}

In [18]:
from tuition_crawl import TuitionCrawl

In [ ]:
# Search each attribute

output_cache = {}
for index, row in target_university_df.iterrows():
    # Convert the row to a dictionary
    row_dict = row.to_dict()
    # The attribute name is used to do something specific
    university_name = row_dict['university_name']
    if pd.isna(university_name):
        break
    print(f"Processing row for university: {university_name}")
    # construct my reference(s)
    reference = []
    if row_dict['website']:
        reference.append(row_dict['website'])
    if row_dict['wikipedia']:
        reference.append(row_dict['wikipedia'])
    for index, attribute_row in attribute_df.iterrows():
        attribute_row = attribute_row.to_dict()
        attribute_name = attribute_row['attribute_name']
        attribute_reference = attribute_row['attribute_reference']
        if pd.isna(attribute_reference):
            attribute_reference = reference
        else:
            attribute_reference += str(reference)
        if pd.isna(attribute_name):
            break
        attribute_name = str(attribute_name)
        if attribute_name == "ID" or attribute_name in row_dict:
            continue
        # if search for tuition, we use crawl instead of the GPT
        if attribute_name in ("domestic_student_tuition", "international_student_tuition"):
            tuition_dict = TuitionCrawl().fetch_tuition(university_name=university_name)
            row_dict[attribute_name] = tuition_dict[attribute_name]
        else:
            # otherwise, we could use GPT to fetch and generate the result
            row_dict[attribute_name], extra_reference = get_value_and_reference_from_gpt(
                university_name=university_name,
                target_attribute=attribute_name,
                format=attribute_row["attribute_format"],
                reference=attribute_reference,
                data_example_pair= university_name + " " + attribute_row['attribute_name'] + " " + str(attribute_row['example']),
                extra_prompt=attribute_row["attribute_prompt"]
            )
        pprint(f"Add {attribute_name} for {university_name}")
    output_cache[university_name] = row_dict
# store the output to the local cache
store_cache(os.path.join(cache_repo_path, "output.json"), pd.DataFrame(output_cache).to_dict())

In [21]:
# save cache to sheet
output_sheet = recreate_sheet(sheet_client=get_sheet_client(), title="output")
output_sheet.append_row(list(expect_column))  # Insert headers

try:
    # Load the cached data into a pandas DataFrame
    with open(os.path.join(cache_repo_path, "output.json"), 'r') as cache_file:
        cache_data = json.load(cache_file)
    
    # If the cached data is a dictionary of dictionaries, convert it to a list of dictionaries
    if isinstance(cache_data, dict):
        cache_data = list(cache_data.values())
    
    # Create a DataFrame from the list of dictionaries
    universities_df = pd.DataFrame(cache_data)
    
    # Cleanse the DataFrame by replacing NaN with empty strings
    universities_df.fillna('', inplace=True)
    
    # Get column headers from the DataFrame
    column_headers = universities_df.columns.tolist()
    
    # Clear the output_sheet and append the column headers
    output_sheet.clear()
    output_sheet.append_row(column_headers)

    # Iterate over DataFrame rows and append each row to the Google Sheet
    for index, row in universities_df.iterrows():
        # Ensure all values are converted to strings, replace None with an empty string
        row_values = row.astype(str).replace({np.nan: None}).tolist()
        output_sheet.append_row(row_values)
except FileNotFoundError:
    print("Cache file not found. Starting with an empty cache.")

Worksheet 'output' found and deleted.
New 'output' worksheet created.
